# Online Simulation-Based Digital Twin: Nibelungenbrücke

This notebook presents an online simulation-based digital twin developed for the Nibelungenbrücke bridge. The system is managed by an orchestration layer that dynamically handles user inputs such as the type of physical model to analyze, the simulation time window, and spatial locations of interest. It supports both standard transient thermal simulations and simulations that incorporate uncertainty quantification (UQ), allowing users to assess not only predicted behavior but also confidence in those predictions. 

The simulations are powered by the FenicSXConcrete package and provide physics-based, real-time insights into the thermomechanical and structural deflection behavior of the bridge.


> ⚠️ **Please run the following cell once before using the notebook to install all required packages in your JupyterHub environment.**  

In [1]:
#import sys
#!{sys.executable} -m pip install git+https://github.com/BAMresearch/FenicsXConcrete pint gmsh pytest jsonschema pandas pyproj tqdm pvlib meshio chaospy==3.3.9


#!{sys.executable} -m pip install git+https://github.com/BAMresearch/FenicsXConcrete@my-grf git+https://github.com/BAMresearch/probeye chaospy==3.3.9 doit==0.36.0 pint==0.24.4 jinja2==3.1.6 pdf2image==1.17.0 pint==0.24.4 pytest==8.4.0 jsonschema==4.24.0 pandas==2.3.0 pyproj tqdm pvlib==0.13.0 meshio python-gmsh==4.12.2 git

This code snippet adjusts the current working directory and updates the system path to ensure proper resolution of local project modules.

In [2]:
import os
import sys

original_cwd = os.getcwd()
root_dir = os.getcwd()
orchestrator_dir = os.path.join(root_dir, 'nibelungenbruecke', 'scripts', 'digital_twin_orchestrator')
os.chdir(orchestrator_dir)
sys.path.insert(0, root_dir)

from nibelungenbruecke.scripts.digital_twin_orchestrator.orchestrator import Orchestrator

### Definition of input parameters

This dictionary defines the configuration for running a digital twin simulation, including model selection, temporal settings, sensor definitions, and simulation options.

In [4]:
simulation_parameters = {
    'simulation_name': 'TestSimulation',
    'model': 'TransientThermal_1',
    'start_time': '2023-08-11T08:00:00Z',
    'end_time': '2023-09-11T16:10:00Z',
    'time_step': '10min',
    'virtual_sensor_positions': [
        {'x': 0.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor1'},
        {'x': 1.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor2'},
        {'x': 1.78, 'y': 0.0, 'z': 26.91, 'name': 'Sensor3'},
        {'x': -1.83, 'y': 0.0, 'z': 0.0, 'name': 'Sensor4'}
    ],
    'plot_pv': True,
    'full_field_results': True, # Set to True if you want full field results, the simulation will take longer and the results will be larger.
    'uncertainty_quantification': False, # Set to True if you want uncertainty quantification, the simulation will take longer and the results will be larger.
}

The `Orchestrator` class serves as the central controller responsible for managing the full digital twin simulation workflow. It orchestrates the configuration, model selection, simulation execution, and optional post-processing steps based on the input parameters defined earlier.

The following line initializes an instance of the `Orchestrator` with the user-defined configuration:

In [5]:
orchestrator = Orchestrator(simulation_parameters)

Provide your key to MKP's API to registers it with the Orchestrator instance:

In [6]:
key=input("\nEnter the code to connect API: ").strip()
#key = ""
orchestrator.set_api_key(key)

Before proceeding with the simulation, the `load()` method verifies whether the coordinates of the virtual sensors lie within the boundaries of the mesh domain.

In [7]:
#orchestrator.load(simulation_parameters)

The simulation is executed using:

In [8]:

results = orchestrator.run()


Info    : Reading '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/models/mesh_3d 1.msh'...
Info    : 2443 entities
Info    : 2197 nodes
Info    : 12920 elements
Info    : Done reading '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/models/mesh_3d 1.msh'
New model 'thermal_model' saved successfully.
Error: The file TransientThermal_1 was not found!
Info    : Reading '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/models/mesh_3d 1.msh'...
Info    : 2443 entities
Info    : 2197 nodes
Info    : 12920 elements
Info    : Done reading '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/models/mesh_3d 1.msh'


### Results
The result at the sensors defined in simulation_parameters are plotted below.

In [ ]:
orchestrator.plot_virtual_sensor_data()

Plot comparison between model response and real sensors when available:

In [ ]:
orchestrator.plot_real_sensor_vs_virtual_sensor()

## Additional results (only if run)

### Full-field response (3D)

The full-field simulation results can be reached with the following method.<br>

! Interactive visualization within the interface is not yet supported but will be introduced soon. Meanwhile files can be downloaded from the specified paths printed below.

In [ ]:
orchestrator.plot_full_field_response(simulation_parameters["full_field_results"])

### Thermal Model Uncertainty Quantification

Uncertainty quantification for the thermal model has been introduced. To enable this feature:  
- Set the parameter `uncertainty_quantification` to `True`  
- Ensure the parameter `plot_pv` is set to `False`  

This configuration activates the uncertainty analysis.

In [ ]:
simulation_parameters = {
    'simulation_name': 'TestSimulation',
    'model': 'TransientThermal_1',
    'start_time': '2023-08-11T08:00:00Z',
    'end_time': '2023-09-13T08:10:00Z',
    'time_step': '10min',
    'virtual_sensor_positions': [
        {'x': 0.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor1'},
        {'x': 1.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor2'},
        {'x': 1.78, 'y': 0.0, 'z': 26.91, 'name': 'Sensor3'},
        {'x': -1.83, 'y': 0.0, 'z': 0.0, 'name': 'Sensor4'}
    ],
    'plot_pv': False,
    'full_field_results': True,
    'uncertainty_quantification': True,
}

Verify that all given virtual sensors fall within the mesh domain and then run the model.

In [ ]:
#orchestrator.load(simulation_parameters)
orchestrator.run(simulation_parameters)

In [ ]:
orchestrator.plot_virtual_sensor_data()

In [ ]:
orchestrator.plot_real_sensor_vs_virtual_sensor()

In [ ]:
orchestrator.plot_full_field_response(simulation_parameters["full_field_results"])

In [ ]:
os.chdir(original_cwd)
print("Working directory restored to:", original_cwd)